<a href="https://colab.research.google.com/github/Sofia-1234567/Forensic-Research-Project-2021/blob/main/Tweets_Collection_for_FRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Twitter properties file contains the Twitter accesstoken, accesstokensecret,apikey and apisecretkey. 
!ls '/content/drive/My Drive/Colab Notebooks/twitterproperties.txt'

'/content/drive/My Drive/Colab Notebooks/twitterproperties.txt'


In [ ]:
!pip install tweepy

In [ ]:
#Tweepy is a Python library that allows us to access Twitter API. 
import tweepy

#Using pandas library, a library for data manipulation and analysis to build a dataframe for the collected tweets
import pandas as pd

In [ ]:
#Reads the properties file from the Google Drive

!pip install ConfigParser

In [ ]:
#In the properties file, the tokens and secret keys are under a section called 'Twitter' which configparser reads. 
import pandas as pd
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitterproperties.txt')

print(config.sections());

['twitter']


In [ ]:
import tweepy as tw

In [ ]:
#Assigns the contents of the token and secret keys from the Twitter section of the properties file to the variables. 
accesstoken=config.get('twitter', 'accesstoken')
accesstokensecret=config.get('twitter', 'accesstokensecret')
apikey=config.get('twitter', 'apikey')
apisecretkey=config.get('twitter', 'apisecretkey')

In [ ]:
#Tweepy AuthHandler authorises the keys and tokens to be accessed 
auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)

#calling on auth and applying a wait limit so the server is not constantly pulling tweets (prevents crashing)
#when the treshold of collected tweets is met, it will refresh in another 15 minutes to pull the next set of tweets
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
#searches for words, and multiple words in one tweet using the OR function
#apply search filters

search_words = 'dog' "-filter:retweets AND filter:images"

#specify date we want the tweets from
date_since = "2020-01-01"


In [ ]:
#tells the tweepy cursor that we want to perform an API search of search words, with language of the tweets being english, and to pull 300 tweets at a time. 
tweets = tw.Cursor(api.search,
                   q=search_words,
                   lang="en",
                   since=date_since, tweet_mode='extended').items(300) 

In [ ]:
#the items that will be extracted per tweet are: unique tweet identifier (as an integer), the full text of the tweet, the date tweet was created, and the URL to the image in attached to the tweet 
tweet_details = [[tweet.id, tweet.full_text, tweet.created_at, tweet.entities['media'][0]['media_url_https']] for tweet in tweets]

In [ ]:
#creating a dataframe of tweet_details
#column headings are: 
tweet_df = pd.DataFrame(data=tweet_details, columns=['tweet_id','tweet_txt', 'created_at', 'tweet_img'])

In [ ]:
#sets the maximum width in characters of a column. When the column overflows, a "..." placeholder is embedded in the output. 
pd.set_option('max_colwidth', 800)

#Displays the first 50 tweets (starting at 0)
tweet_df.head(10)

,tweet_id,tweet_txt,created_at,tweet_img
0,1373618101430853632,"Wait, I was today old when I learned that Osaki Hachimangu has a connection to people born on the year of the dog and boar. Guess who's born on the year of the dog? 😁😁😁 https://t.co/VHJbS096V9",2021-03-21 12:50:50,https://pbs.twimg.com/media/ExASYSvVkAIg3Za.jpg
1,1373618015057670149,*sad dog noises* https://t.co/iiFK2OORrx,2021-03-21 12:50:30,https://pbs.twimg.com/media/ExASElZXEAYr_T3.jpg
2,1373617965900443648,Random things my dog is afraid of: https://t.co/hOOyQZXDCS,2021-03-21 12:50:18,https://pbs.twimg.com/media/ExASQl1WQAAVYtV.jpg
3,1373617920044044289,Feeling Proud....Yesterday we completed our 100km March dog walking challenge in aid of @DementiaUK . Mo and I will keep walking and adding to the kilometres until the end of March and hopefully add to the £950 that we have raised so far. 🏆 https://t.co/hG624KECeT,2021-03-21 12:50:07,https://pbs.twimg.com/media/ExASNqBXMAE7lTb.jpg
4,1373617889190772736,It’s my other Big Dog Birthday. 🎂🎊🎁#NoSocialsMarlin https://t.co/CXw692x6yL,2021-03-21 12:50:00,https://pbs.twimg.com/media/ExASMHzWQAEwP_v.jpg
5,1373617809536737280,"4. I have two doggos, both Staffordshire bull terriers. They are Tyson (My dog), he's almost 10 (first pic) and Bobby (Family dog) almost 11 (second pic). They follow me around everywhere like I'm a pack leader and I love them very much 😊 They're right here with me now (3rd pic) https://t.co/4wGsiqDUxw",2021-03-21 12:49:41,https://pbs.twimg.com/media/ExASFh5WUAUXQs6.jpg
6,1373617786451279876,"Walking the dog this morning used my usual shortcut and almost pooped my pants, because thought this guy was real https://t.co/ykMrlOF4Kd",2021-03-21 12:49:35,https://pbs.twimg.com/media/ExASGHhWUAAnHwd.jpg
7,1373617726787350528,"Hidden away in Battersea Park is this memorial to the Brown Dog. And look, someone has left it a stick!\n\nThe original brown dog caused riots after allegations of illegal and inhumane experimentation at University College London. https://t.co/5ACHvzpyAa",2021-03-21 12:49:21,https://pbs.twimg.com/media/ExASB45XMAMJS3J.jpg
8,1373617704695828483,"Udupi: Leopard enters home while chasing pet dog, trapped inside\n\n► Watch Video\n\n@aranya_kfd \n\nClick 👉 https://t.co/rU5iu0T1xJ https://t.co/boIU47JhQa",2021-03-21 12:49:16,https://pbs.twimg.com/media/ExAR962VoAAGUY6.jpg
9,1373617699331444740,Gratuitous dog pic while waiting for breakfast to cook... https://t.co/HELYpVgpyI,2021-03-21 12:49:14,https://pbs.twimg.com/media/ExAR-XHXEAEHtJH.jpg


In [ ]:
#saves the dataframe to .csv file
#tweet_df.to_csv('data.csv')